# Data exploration

### Explore bronze database and show what tables are present

In [1]:
import sqlite3
import pandas as pd
from collections import defaultdict

# Connect to the bronze database
conn = sqlite3.connect('../data/1_bronze/bronze_data.db')
cursor = conn.cursor()

# Get list of all tables in bronze database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Group tables by prefix
table_groups = defaultdict(list)
for table in tables:
    table_name = table[0]
    prefix = table_name.split('_', 1)[0]  # Extract prefix before first underscore
    table_groups[prefix].append(table_name)

# Display grouped tables
print("Tables grouped by prefix:")
for prefix, table_list in sorted(table_groups.items()):
    print(f"\n{prefix}:")
    for t in table_list:
        print(f"  - {t}")

Tables grouped by prefix:

80072ned:
  - 80072ned_dim_BedrijfskenmerkenSBI2008
  - 80072ned_dim_CategoryGroups
  - 80072ned_dim_DataProperties
  - 80072ned_dim_Perioden
  - 80072ned_dim_TableInfos
  - 80072ned_fact

83157NED:
  - 83157NED_dim_BedrijfstakkenBranchesSBI2008
  - 83157NED_dim_CategoryGroups
  - 83157NED_dim_DataProperties
  - 83157NED_dim_Marges
  - 83157NED_dim_Perioden
  - 83157NED_dim_TableInfos
  - 83157NED_fact

83415NED:
  - 83415NED_dim_DataProperties
  - 83415NED_dim_Marges
  - 83415NED_dim_Perioden
  - 83415NED_dim_TableInfos
  - 83415NED_fact

sample:
  - sample_org


In [2]:
# Get all table names starting with '80072ned'
tables_80072ned = table_groups['80072ned']

# Load each table into a DataFrame and store in a dictionary
dfs_80072ned = {table: pd.read_sql_query(f"SELECT * FROM [{table}]", conn) for table in tables_80072ned}

# Show the keys (table names) and preview the first table
print("Loaded tables:", list(dfs_80072ned.keys()))
dfs_80072ned[tables_80072ned[0]].head()

Loaded tables: ['80072ned_dim_BedrijfskenmerkenSBI2008', '80072ned_dim_CategoryGroups', '80072ned_dim_DataProperties', '80072ned_dim_Perioden', '80072ned_dim_TableInfos', '80072ned_fact']


,bronze_pk,Key,Title,Description,CategoryGroupID,_source_file
0,BedrijfskenmerkenSBI2008.json_T001081,T001081,A-U Alle economische activiteiten,Alle economische activiteiten \r\nDeze categor...,1,BedrijfskenmerkenSBI2008.json
1,BedrijfskenmerkenSBI2008.json_301000,301000,"A Landbouw, bosbouw en visserij","Landbouw, bosbouw en visserij \r\nDeze sectie ...",2,BedrijfskenmerkenSBI2008.json
2,BedrijfskenmerkenSBI2008.json_305700,305700,B Delfstoffenwinning,Winning van delfstoffen \r\nDeze sectie omvat:...,3,BedrijfskenmerkenSBI2008.json
3,BedrijfskenmerkenSBI2008.json_300003,300003,B-F Nijverheid en energie,Nijverheid en energie \r\nDeze categorie is ee...,2,BedrijfskenmerkenSBI2008.json
4,BedrijfskenmerkenSBI2008.json_307500,307500,C Industrie,Industrie \r\nDeze sectie omvat: \r\n- de mech...,3,BedrijfskenmerkenSBI2008.json


### Read all content from the 80072ned tables in database bronze_data and merge the tables on keys.

In [3]:
# Get fact table from the loaded dataframes
fact_df = dfs_80072ned['80072ned_fact']

# Get the dimension tables
periode_df = dfs_80072ned['80072ned_dim_Perioden']
bedrijfs_df = dfs_80072ned['80072ned_dim_BedrijfskenmerkenSBI2008']
category_df = dfs_80072ned['80072ned_dim_CategoryGroups']

# Perform the joins
enriched_fact = fact_df.merge(periode_df, left_on='Perioden', right_on='Key', how='left', suffixes=('', '_periode'))
enriched_fact = enriched_fact.merge(bedrijfs_df, left_on='BedrijfskenmerkenSBI2008', right_on='Key', how='left', suffixes=('', '_bedrijfs'))
enriched_fact = enriched_fact.merge(category_df, left_on='CategoryGroupID', right_on='ID', how='left', suffixes=('', '_category'))

# # Select and rename columns
enriched_fact = enriched_fact[['ID', 'BedrijfskenmerkenSBI2008', 'Perioden', 'Ziekteverzuimpercentage_1' ,'Title', 'Description', 'Status', 'Key_bedrijfs', 'Title_bedrijfs', 'Description_bedrijfs', 'CategoryGroupID', 'ID_category', 'DimensionKey', 'Title_category', 'Description_category', 'ParentID']]
# enriched_fact.columns = ['fact_ID', 'Ziekteverzuimpercentage_1', 'periode_key', 'periode_title', 'periode_description', 'periode_status', 'bedrijfs_key', 'bedrijfs_title', 'bedrijfs_description', 'CategoryGroupID', 'category_id', 'category_dimension_key', 'category_title', 'category_description', 'category_parent_id']

print(enriched_fact.columns)
print(enriched_fact.head())

# # Close the database connection
conn.close()

Index(['ID', 'BedrijfskenmerkenSBI2008', 'Perioden',
       'Ziekteverzuimpercentage_1', 'Title', 'Description', 'Status',
       'Key_bedrijfs', 'Title_bedrijfs', 'Description_bedrijfs',
       'CategoryGroupID', 'ID_category', 'DimensionKey', 'Title_category',
       'Description_category', 'ParentID'],
      dtype='object')
   ID BedrijfskenmerkenSBI2008  Perioden  Ziekteverzuimpercentage_1  \
0   0                  T001081  1996KW01                        5.5   
1   1                  T001081  1996KW02                        4.6   
2   2                  T001081  1996KW03                        4.0   
3   3                  T001081  1996KW04                        4.7   
4   4                  T001081  1996JJ00                        4.7   

              Title Description      Status Key_bedrijfs  \
0  1996 1e kwartaal              Definitief      T001081   
1  1996 2e kwartaal              Definitief      T001081   
2  1996 3e kwartaal              Definitief      T001081   
3  1

### Which variables are numerical? And which are strings? How many variables do we have of both types? How many observations do we have? Suggestion: Split the original data in a data frame containing the numerical data and a data frame containing the string data.

**numerical data:**

The numerical type allows mathematical operations.

In [4]:
# Using Pandas.
df_pd_enriched_num    = enriched_fact.select_dtypes(include='number')

# Column names (Pandas: df_pd_orig_num.columns.tolist()).
l_df_num_names = df_pd_enriched_num.columns

print(f"We have {len(l_df_num_names)} numerical variables:\n{l_df_num_names}")

We have 4 numerical variables:
Index(['ID', 'Ziekteverzuimpercentage_1', 'CategoryGroupID', 'ID_category'], dtype='object')


In [5]:
# In Pandas.
df_pd_enriched_str    = enriched_fact.select_dtypes(include='object')

# Column names (Pandas: df_pd_orig_str.columns.tolist()).
l_df_str_names = df_pd_enriched_str.columns

print(f"We have {len(l_df_str_names)} string variables:\n{l_df_str_names}")

We have 12 string variables:
Index(['BedrijfskenmerkenSBI2008', 'Perioden', 'Title', 'Description',
       'Status', 'Key_bedrijfs', 'Title_bedrijfs', 'Description_bedrijfs',
       'DimensionKey', 'Title_category', 'Description_category', 'ParentID'],
      dtype='object')


Do we have all columns in both data frames? Yes, we do!

In [6]:
print(
    f"The original data has {enriched_fact.shape[1]} columns. The number and string data "
    f"have total of {df_pd_enriched_num.shape[1] + df_pd_enriched_str.shape[1]} columns."
)

The original data has 16 columns. The number and string data have total of 16 columns.


We observe 5772 observations (rows) in the data frame.

In [7]:
print(f"Number of observations: {enriched_fact.shape[0]}")

Number of observations: 5772


In [8]:
# We use the number of observations more often, so we define a variable.
n_enriched_obs = enriched_fact.shape[0]

### How many missing values do each of the variables have (variable completeness) and what are the variable types? Is `Ziekteverzuimpercentage_1` complete? And what other, missing-like values do you observe in the string data? Create a frequency table of missing data per variable.

**numerical data**:

Pandas' `describe()` function outputs some descriptive statistics. For numerical data these stats include: count of non-null values, mean, standard deviation, range (i.e., min and max), the lower quartile, the median, and the upper quartile. What can we conclude for the 'SalePrice' variable (min, max, median vs average)?

**numerical data:**

Not all numerical variables are complete. For example, `LotFrontage` has 490 missing values. All numerical variables are of type "int64".


In [9]:
# Pandas.
df_pd_missing_data_num = (
    
    pd.DataFrame({

        'data_type':    df_pd_enriched_num.dtypes,
        'missing':      df_pd_enriched_num.isnull().sum()   
    })

    # Add complete%
    .assign(
        complete_pct = lambda x: round(100 * (n_enriched_obs - x['missing']) / n_enriched_obs, 2)
    )

    # Remove variables that are complete.
    .query("complete_pct < 100")

    # Sort table by data completeness in descending order.
    .sort_values(by = 'complete_pct')
)

print(df_pd_missing_data_num)
print("")


                          data_type  missing  complete_pct
Ziekteverzuimpercentage_1   float64      310         94.63



**String data:**

Are all string variables are complete? Let's investigate the unique values. In case of Pandas we observe `nan` in addition to non-null data. 

In [19]:
# Pandas
print("Pandas:")

for name in l_df_str_names:
    print(f"{name}: {df_pd_enriched_str[name].unique()}")

Pandas:
BedrijfskenmerkenSBI2008: ['T001081' '301000' '305700' '300003' '307500' '307610' '317105' '320005'
 '328110' '346600' '348000' '350000' '300007' '354200' '354300' '356900'
 '371600' '383100' '383200' '389100' '391600' '396300' '402000' '403300'
 '410200' '415300' '300013' '417400' '419000' '422400' '422500' '422600'
 '425300' '426600' '428100' '435500' 'WP19078' 'WP19091' 'WP19098']
Perioden: ['1996KW01' '1996KW02' '1996KW03' '1996KW04' '1996JJ00' '1997KW01'
 '1997KW02' '1997KW03' '1997KW04' '1997JJ00' '1998KW01' '1998KW02'
 '1998KW03' '1998KW04' '1998JJ00' '1999KW01' '1999KW02' '1999KW03'
 '1999KW04' '1999JJ00' '2000KW01' '2000KW02' '2000KW03' '2000KW04'
 '2000JJ00' '2001KW01' '2001KW02' '2001KW03' '2001KW04' '2001JJ00'
 '2002KW01' '2002KW02' '2002KW03' '2002KW04' '2002JJ00' '2003KW01'
 '2003KW02' '2003KW03' '2003KW04' '2003JJ00' '2004KW01' '2004KW02'
 '2004KW03' '2004KW04' '2004JJ00' '2005KW01' '2005KW02' '2005KW03'
 '2005KW04' '2005JJ00' '2006KW01' '2006KW02' '2006KW03' '20

### Conduct descriptive/summary statistics for numerical variables (e.g., mean, median, std, and range) and for string variables (e.g., number of unique values, mode, and their frequency)

**numerical data**:

Pandas' `describe()` function outputs some descriptive statistics. For numerical data these stats include: count of non-null values, mean, standard deviation, range (i.e., min and max), the lower quartile, the median, and the upper quartile. What can we conclude for the 'SalePrice' variable (min, max, median vs average)?

In [20]:
df_pd_enriched_num.describe()

,ID,Ziekteverzuimpercentage_1,CategoryGroupID,ID_category
count,5772.000000,5462.000000,5772.000000,5772.000000
mean,2885.500000,4.580264,3.333333,3.333333
std,1666.377208,1.299420,0.857421,0.857421
min,0.000000,1.400000,1.000000,1.000000
25%,1442.750000,3.600000,3.000000,3.000000
50%,2885.500000,4.500000,3.000000,3.000000
75%,4328.250000,5.400000,4.000000,4.000000
max,5771.000000,10.100000,5.000000,5.000000


**String data:**

For string data, Pandas' `describe()` function includes: count of non-null values and unique values, value at high frequency ('top'), and its concerned frequency. 

In [21]:
df_pd_enriched_str.describe()


,BedrijfskenmerkenSBI2008,Perioden,Title,Description,Status,Key_bedrijfs,Title_bedrijfs,Description_bedrijfs,DimensionKey,Title_category,Description_category,ParentID
count,5772,5772,5772,5733,5772,5772,5772,5772,5772,5772,148,5772
unique,39,148,148,2,2,39,39,37,1,5,1,1
top,T001081,1996KW01,1996 1e kwartaal,,Definitief,T001081,A-U Alle economische activiteiten,"Het aantal ""werkzame personen"" bestaat uit: \r...",BedrijfskenmerkenSBI2008,Bedrijfstak,,0
freq,148,39,39,5616,5655,148,148,444,5772,2664,148,5772


### Save data to pickle file

We save objects - that we need in subsequent exercises - to a pickle file.

In [26]:
import os
import pickle

# Create dictionary 'dc_80072ned' with objects that will be used in the next exercises.
dc_uwv_80072ned = {
    'enriched_fact': enriched_fact,
    'l_df_num_names': l_df_num_names,
    'l_df_str_names': l_df_str_names,
}

# Ensure directory exists then save dc_uwv_80072ned as 'dc-uwv-80072ned.pkl'
out_dir = '../data/1_bronze'
os.makedirs(out_dir, exist_ok=True)
file_path = os.path.join(out_dir, 'dc-uwv-80072ned.pkl')

if os.path.exists(file_path):
    print(f"Overwriting existing file: {file_path}")
else:
    print(f"Saving to: {file_path}")

with open(file_path, 'wb') as pickle_file:
    pickle.dump(dc_uwv_80072ned, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

Saving to: ../data/1_bronze\dc-uwv-80072ned.pkl
